# 🍏 Observability & Tracing Demo with `azure-ai-projects` and `azure-ai-inference` 🍎

Welcome to this **Health & Fitness**-themed notebook, where we'll explore how to set up **observability** and **tracing** for:

1. **Basic LLM calls** using an `AIProjectClient`.
2. **Multi-step** interactions using an **Agent** (such as a Health Resource Agent).
3. **Tracing** your local usage in **console** (stdout) or via an **OTLP endpoint** (like **Prompty** or **Aspire**).
4. Sending those **traces** to **Azure Monitor** (Application Insights) so you can view them in **Azure AI Foundry**.

> **Disclaimer**: This is a fun demonstration of AI and observability! Any references to workouts, diets, or health routines in the code or prompts are purely for **educational** purposes. Always consult a professional for health advice.

## Contents
1. **Initialization**: Setting up environment, creating clients.
2. **Basic LLM Call**: Quick demonstration of retrieving model completions.
3. **Connections**: Listing project connections.
4. **Observability & Tracing**
   - **Console / Local** tracing
   - **Prompty / Aspire**: piping traces to a local OTLP endpoint
   - **Azure Monitor** tracing: hooking up to Application Insights
   - **Verifying** your traces in Azure AI Foundry
5. **Agent-based Example**:
   - Creating a simple "Health Resource Agent" referencing sample docs.
   - Multi-turn conversation with tracing.
   - Cleanup.

<img src="./seq-diagrams/1-observability.png" width="50%"/>

## 1. Initialization & Setup
**Prerequisites**:
- A `.env` file containing `PROJECT_CONNECTION_STRING` (and optionally `MODEL_DEPLOYMENT_NAME`).
- Roles/permissions in Azure AI Foundry that let you do inference & agent creation.
- A local environment with `azure-ai-projects`, `azure-ai-inference`, `opentelemetry` packages installed.

**What we do**:
- Load environment variables.
- Initialize `AIProjectClient`.
- Check that we can talk to a model (like `gpt-4o`).

In [1]:
import os
import sys
import time
from pathlib import Path
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.inference.models import UserMessage, CompletionsFinishReason

# Load environment variables
notebook_path = Path().absolute()
env_path = notebook_path.parent.parent / '.env'  # Adjust path as needed
load_dotenv(env_path)

connection_string = os.environ.get("PROJECT_CONNECTION_STRING")
if not connection_string:
    raise ValueError("🚨 PROJECT_CONNECTION_STRING not set in .env.")

# Initialize AIProjectClient
try:
    project_client = AIProjectClient.from_connection_string(
        credential=DefaultAzureCredential(),
        conn_str=connection_string
    )
    print("✅ Successfully created AIProjectClient!")
except Exception as e:
    print(f"❌ Error creating AIProjectClient: {e}")

✅ Successfully created AIProjectClient!


## 2. Basic LLM Call
We'll do a **quick** chat completion request to confirm everything is working. We'll ask a simple question: "How many feet are in a mile?"

In [2]:
try:
    # Create a ChatCompletions client
    inference_client = project_client.inference.get_chat_completions_client()
    # Default to "gpt-4o" if no env var is set
    model_name = os.environ.get("MODEL_DEPLOYMENT_NAME", "gpt-4o")

    user_question = "How many feet are in a mile?"
    response = inference_client.complete(
        model=model_name,
        messages=[UserMessage(content=user_question)]
    )
    print("\n💡Response:")
    print(response.choices[0].message.content)
    print("\nFinish reason:", response.choices[0].finish_reason)

except Exception as e:
    print("❌ Could not complete the chat request:", e)


💡Response:
There are **5,280 feet** in a mile.

Finish reason: CompletionsFinishReason.STOPPED


## 3. List & Inspect Connections
Check out the **connections** your project has: these might be Azure OpenAI or other resource attachments. We'll just list them here for demonstration.

In [3]:
from azure.ai.projects.models import ConnectionType

all_conns = project_client.connections.list()
print(f"🔎 Found {len(all_conns)} total connections.")
for idx, c in enumerate(all_conns):
    print(f"{idx+1}) Name: {c.name}, Type: {c.connection_type}, Endpoint: {c.endpoint_url}")

# Filter for Azure OpenAI connections
aoai_conns = project_client.connections.list(connection_type=ConnectionType.AZURE_OPEN_AI)
print(f"\n🌀 Found {len(aoai_conns)} Azure OpenAI connections:")
for c in aoai_conns:
    print(f"   -> {c.name}")

# Get default connection of type AZURE_AI_SERVICES
default_conn = project_client.connections.get_default(connection_type=ConnectionType.AZURE_AI_SERVICES,
                                                     include_credentials=False)
if default_conn:
    print("\n⭐ Default Azure AI Services connection:")
    print(default_conn)
else:
    print("No default connection found for Azure AI Services.")

🔎 Found 6 total connections.
1) Name: ai-vanhub14322613050517_aoai, Type: ConnectionType.AZURE_OPEN_AI, Endpoint: https://ai-vanhub14322613050517.openai.azure.com
2) Name: ai-vanhub14322613050517, Type: ConnectionType.AZURE_AI_SERVICES, Endpoint: https://ai-vanhub14322613050517.cognitiveservices.azure.com
3) Name: vancouveraisearch, Type: ConnectionType.AZURE_AI_SEARCH, Endpoint: https://vancouveraisearch.search.windows.net
4) Name: vancouverbing, Type: ConnectionType.API_KEY, Endpoint: https://api.bing.microsoft.com
5) Name: vancouver/workspaceartifactstore, Type: ConnectionType.AZURE_BLOB_STORAGE, Endpoint: https://stvanhub14322613050517.core.windows.net/9b21d268-39c4-4a54-a8a6-da3bc39c1cbe-azureml
6) Name: vancouver/workspaceblobstore, Type: ConnectionType.AZURE_BLOB_STORAGE, Endpoint: https://stvanhub14322613050517.core.windows.net/9b21d268-39c4-4a54-a8a6-da3bc39c1cbe-azureml-blobstore

🌀 Found 1 Azure OpenAI connections:
   -> ai-vanhub14322613050517_aoai

⭐ Default Azure AI Servi

# 4. Observability & Tracing

We want to **collect telemetry** from our LLM calls, for example:
- Timestamps of requests.
- Latency.
- Potential errors.
- Optionally, the actual prompts & responses (if you enable content recording).

We'll show how to set up:
1. **Console** or local OTLP endpoint instrumentation.
2. **Azure Monitor** instrumentation with Application Insights.
3. **Viewing** your traces in Azure AI Foundry's portal.

## 4.1 Local Console Debugging
We'll install instrumentation packages and enable them. Then we'll do a quick chat call to see if logs appear in **stdout**.

**Note**: If you want to see more advanced local dashboards, you can:
- Use [Prompty](https://github.com/microsoft/prompty).
- Use [Aspire Dashboard](https://learn.microsoft.com/dotnet/aspire/fundamentals/dashboard/standalone?tabs=bash) to visualize your OTLP traces.

In [4]:
# You only need to install these once.
!pip install opentelemetry-instrumentation-openai-v2 opentelemetry-exporter-otlp-proto-grpc

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/4.5 MB ? eta -:--:--
   -------------- ------------------------- 1.6/4.5 MB 8.6 MB/s eta 0:00:01
   ------------------------------------- -- 4.2/4.5 MB 10.6 MB/s eta 0:00:01
   ---------------------------------------- 4.5/4.5 MB 10.2 MB/s  0:00:00

   -- -------------------------------------  1/16 [wrapt]
   ------- --------------------------------  3/16 [protobuf]
   ------- --------------------------------  3/16 [protobuf]
   ------- --------------------------------  3/16 [protobuf]
   ------- --------------------------------  3/16 [protobuf]
   ------- --------------------------------  3/16 [protobuf]
   ---------- -----------------------------  4/16 [packaging]
   ------------ ---------------------------  5/16 [grpcio]
   ------------ ---------------------------  5/16 [grpcio]
   ------------ ---------------------------  5/16 [grpcio]
   ------------ -------

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


### 4.1.1 Enable OpenTelemetry for Azure AI Inference
We set environment variables to ensure:
1. **Prompt content** is captured (optional!)
2. The **Azure SDK** uses OpenTelemetry as its tracing implementation.
3. We call `AIInferenceInstrumentor().instrument()` to patch and enable the instrumentation.


In [5]:
import os
from azure.ai.inference.tracing import AIInferenceInstrumentor

# (Optional) capture prompt & completion contents in traces
os.environ["AZURE_TRACING_GEN_AI_CONTENT_RECORDING_ENABLED"] = "true"  # or 'false'

# Let the Azure SDK know we want to use OpenTelemetry
os.environ["AZURE_SDK_TRACING_IMPLEMENTATION"] = "opentelemetry"

# Instrument the Azure AI Inference client library
AIInferenceInstrumentor().instrument()
print("✅ Azure AI Inference instrumentation enabled.")

✅ Azure AI Inference instrumentation enabled.


### 4.1.2 Point Traces to Console or Local OTLP
The simplest is to pipe them to **stdout**. If you want to send them to **Prompty** or **Aspire**, specify the local OTLP endpoint URL (usually `"http://localhost:4317"` or similar).

In [9]:
project_client.telemetry.enable(destination=sys.stdout)
# Or, to send to a local OTLP collector (Prompty/Aspire), do:
#   project_client.telemetry.enable(destination="http://localhost:4317")

try:
    local_client = project_client.inference.get_chat_completions_client()
    user_prompt = "What's a simple 5-minute warmup routine?"
    local_resp = local_client.complete(
        model=os.environ.get("MODEL_DEPLOYMENT_NAME", "gpt-4o"),
        messages=[UserMessage(content=user_prompt)]
    )
    print("\n🤖 Response:", local_resp.choices[0].message.content)
except Exception as exc:
    print(f"❌ Error in local-tracing example: {exc}")


🤖 Response: Sure! Here’s a simple **5-minute warmup routine** to get your body ready for exercise:

1. **March or Jog in Place** – 1 minute  
   Get your heart rate up and start moving.

2. **Arm Circles** – 1 minute  
   Stand tall and swing your arms in big circles, 30 seconds forward, 30 seconds backward.

3. **Bodyweight Squats** – 1 minute  
   Feet shoulder-width apart, squat down and stand up. Go at a steady pace.

4. **Torso Twists** – 1 minute  
   Stand with feet hip-width apart, twist your torso side to side, letting your arms swing loosely.

5. **High Knees or Jumping Jacks** – 1 minute  
   Finish strong to get your blood flowing. Do high knees (lifting each knee up as high as possible), or do jumping jacks.

**Tip:** Breathe naturally, move gently, and feel free to modify any movement to suit your body!


## 4.2 Azure Monitor Tracing (Application Insights)
Now we'll set up tracing to **Application Insights**, which will forward your logs to the **Azure AI Foundry** **Tracing** page.

**Steps**:
1. In AI Foundry, go to your project’s **Tracing** tab, attach (or create) an **Application Insights** resource.
2. In code, call `project_client.telemetry.get_connection_string()` to retrieve the instrumentation key.
3. Use `azure.monitor.opentelemetry.configure_azure_monitor(...)` with that connection.
4. Make an inference call -> logs appear in the Foundry portal (and in Azure Monitor itself).


In [10]:
from azure.monitor.opentelemetry import configure_azure_monitor
from azure.ai.inference.models import UserMessage

app_insights_conn_str = project_client.telemetry.get_connection_string()
if app_insights_conn_str:
    print("🔧 Found App Insights connection string, configuring...")
    configure_azure_monitor(connection_string=app_insights_conn_str)
    # Optionally add more instrumentation (for openai or langchain):
    project_client.telemetry.enable()
    
    # Let's do a test call that logs to AI Foundry's Tracing page
    try:
        with project_client.inference.get_chat_completions_client() as client:
            prompt_msg = "Any easy at-home cardio exercise recommendations?"
            response = client.complete(
                model=os.environ.get("MODEL_DEPLOYMENT_NAME", "gpt-4o"),
                messages=[UserMessage(content=prompt_msg)]
            )
            print("\n🤖 Response (logged to App Insights):")
            print(response.choices[0].message.content)
    except Exception as e:
        print("❌ Chat completions with Azure Monitor example failed:", e)
else:
    print("No Application Insights connection string is configured in this project.")

🔧 Found App Insights connection string, configuring...

🤖 Response (logged to App Insights):
Absolutely! Here are some easy at-home cardio exercises that require no equipment (just some space and comfortable shoes):

**1. Jumping Jacks:**  
Classic and effective. Do them in sets of 30-60 seconds.

**2. High Knees:**  
Jog in place, lifting your knees as high as possible.

**3. Marching in Place:**  
A lower-impact alternative to high knees. Pump your arms to raise your heart rate.

**4. Mountain Climbers:**  
Start in a plank position and alternate bringing knees toward your chest.

**5. Dancing:**  
Put on your favorite song and dance around! Great cardio and mood booster.

**6. Step Touch:**  
Step side to side, touching one foot next to the other. Swing your arms for extra movement.

**7. Butt Kicks:**  
Jog or march in place, aiming to kick your heels toward your glutes.

**8. Fast Feet:**  
Stand with knees slightly bent, and quickly tap your feet on the ground.

**9. Shadow Boxin

### 4.3 Viewing Traces in Azure AI Foundry
After running the above code:
1. Go to your AI Foundry project.
2. Click **Tracing** on the sidebar.
3. You should see the logs from your calls.
4. Filter, expand, or explore them as needed.

Also, if you want more advanced dashboards, you can open your **Application Insights** resource from the Foundry. In the App Insights portal, you get additional features like **end-to-end transaction** details, query logs, etc.


# 5. Agent-based Example
We'll now create a **Health Resource Agent** that references sample docs about recipes or guidelines, then demonstrate:
1. Creating an Agent with instructions.
2. Creating a conversation thread.
3. Running multi-step queries with **observability** enabled.
4. Optionally cleaning up resources at the end.

> The agent approach is helpful when you want more sophisticated conversation flows or **tool usage** (like file search).

## 5.1 Create Sample Files & Vector Store
We'll create dummy `.md` files about recipes/guidelines, then push them into a **vector store** so our agent can do semantic search.

(*This portion is a quick summary—see [the other file-search tutorial] if you need more details.)

In [11]:
from azure.ai.projects.models import (
    FileSearchTool,
    FilePurpose,
    MessageTextContent,
    MessageRole
)

def create_sample_files():
    """Create some local .md files with sample text."""
    recipes_md = (
        """# Healthy Recipes Database\n\n"
        "## Gluten-Free Recipes\n"
        "1. Quinoa Bowl\n"
        "   - Ingredients: quinoa, vegetables, olive oil\n"
        "   - Instructions: Cook quinoa, add vegetables\n\n"
        "2. Rice Pasta\n"
        "   - Ingredients: rice pasta, mixed vegetables\n"
        "   - Instructions: Boil pasta, sauté vegetables\n\n"
        "## Diabetic-Friendly Recipes\n"
        "1. Low-Carb Stir Fry\n"
        "   - Ingredients: chicken, vegetables, tamari sauce\n"
        "   - Instructions: Cook chicken, add vegetables\n\n"
        "## Heart-Healthy Recipes\n"
        "1. Baked Salmon\n"
        "   - Ingredients: salmon, lemon, herbs\n"
        "   - Instructions: Season salmon, bake\n\n"
        "2. Mediterranean Bowl\n"
        "   - Ingredients: chickpeas, vegetables, tahini\n"
        "   - Instructions: Combine ingredients\n"""
    )

    guidelines_md = (
        """# Dietary Guidelines\n\n"
        "## General Guidelines\n"
        "- Eat a variety of foods\n"
        "- Control portion sizes\n"
        "- Stay hydrated\n\n"
        "## Special Diets\n"
        "1. Gluten-Free Diet\n"
        "   - Avoid wheat, barley, rye\n"
        "   - Focus on naturally gluten-free foods\n\n"
        "2. Diabetic Diet\n"
        "   - Monitor carbohydrate intake\n"
        "   - Choose low glycemic foods\n\n"
        "3. Heart-Healthy Diet\n"
        "   - Limit saturated fats\n"
        "   - Choose lean proteins\n"""
    )

    with open("recipes.md", "w", encoding="utf-8") as f:
        f.write(recipes_md)
    with open("guidelines.md", "w", encoding="utf-8") as f:
        f.write(guidelines_md)

    print("📄 Created sample resource files: recipes.md, guidelines.md")
    return ["recipes.md", "guidelines.md"]

sample_files = create_sample_files()

def create_vector_store(files, store_name="my_health_resources"):
    try:
        uploaded_ids = []
        for fp in files:
            upl = project_client.agents.upload_file_and_poll(
                file_path=fp,
                purpose=FilePurpose.AGENTS  # Add FilePurpose.AGENTS here
            )
            uploaded_ids.append(upl.id)
            print(f"✅ Uploaded: {fp} -> File ID: {upl.id}")

        # Create vector store from these file IDs
        vs = project_client.agents.create_vector_store_and_poll(
            file_ids=uploaded_ids,
            name=store_name
        )
        print(f"🎉 Created vector store '{store_name}', ID: {vs.id}")
        return vs, uploaded_ids
    except Exception as e:
        print(f"❌ Error creating vector store: {e}")
        return None, []

vector_store, file_ids = None, []
if sample_files:
    vector_store, file_ids = create_vector_store(sample_files, store_name="health_resources_example")

📄 Created sample resource files: recipes.md, guidelines.md
✅ Uploaded: recipes.md -> File ID: assistant-C4gjJRMXVJmiSJEBnMmTjr
✅ Uploaded: guidelines.md -> File ID: assistant-KCGZ3FDTuZzALmSJJq7WUY
🎉 Created vector store 'health_resources_example', ID: vs_YE8bAMpDuze4sisdvFJWE3vd


## 5.2 Create a Health Resource Agent
We'll create a **FileSearchTool** referencing the vector store, then create an agent with instructions that it should:
1. Provide disclaimers.
2. Offer general nutrition or recipe tips.
3. Cite sources if possible.
4. Encourage professional consultation for deeper medical advice.


In [12]:
from azure.ai.projects.models import FileSearchTool, FilePurpose
from azure.ai.projects.models import ConnectionType, MessageTextContent, MessageRole

def create_health_agent(vs_id):
    try:
        # The tool references our vector store so the agent can search it
        file_search_tool = FileSearchTool(vector_store_ids=[vs_id])
        
        instructions = """
            You are a health resource advisor with access to dietary and recipe files.
            You:
            1. Always present disclaimers (you're not a medical professional)
            2. Provide references to files when possible
            3. Focus on general nutrition or recipe tips.
            4. Encourage professional consultation for more detailed advice.
        """

        agent = project_client.agents.create_agent(
            model=os.environ.get("MODEL_DEPLOYMENT_NAME", "gpt-4o"),
            name="health-search-agent",
            instructions=instructions,
            tools=file_search_tool.definitions,
            tool_resources=file_search_tool.resources
        )
        print(f"🎉 Created agent '{agent.name}' with ID: {agent.id}")
        return agent
    except Exception as e:
        print(f"❌ Error creating health agent: {e}")
        return None

health_agent = None
if vector_store:
    health_agent = create_health_agent(vector_store.id)

🎉 Created agent 'health-search-agent' with ID: asst_ifb9vnfjBZk7awt5vXyHCujV


## 5.3 Using the Agent
Let's create a new conversation **thread** and ask the agent some questions. We'll rely on the **observability** settings we already configured so each step is traced.


In [13]:
def create_thread():
    try:
        thread = project_client.agents.create_thread()
        print(f"📝 Created new thread, ID: {thread.id}")
        return thread
    except Exception as e:
        print(f"❌ Could not create thread: {e}")
        return None

def ask_question(thread_id, agent_id, user_question):
    try:
        # 1) Add user message
        msg = project_client.agents.create_message(
            thread_id=thread_id,
            role="user",
            content=user_question
        )
        print(f"User asked: '{user_question}'")
        # 2) Create & process a run
        run = project_client.agents.create_and_process_run(
            thread_id=thread_id,
            agent_id=agent_id
        )
        print(f"Run finished with status: {run.status}")
        if run.last_error:
            print("Error details:", run.last_error)
        return run
    except Exception as e:
        print(f"❌ Error asking question: {e}")
        return None

if health_agent:
    thread = create_thread()
    if thread:
        # Let's ask a few sample questions
        queries = [
            "Could you suggest a gluten-free lunch recipe?",
            "Show me some heart-healthy meal ideas.",
            "What guidelines do you have for someone with diabetes?"
        ]
        for q in queries:
            ask_question(thread.id, health_agent.id, q)


📝 Created new thread, ID: thread_x5AHqTMGKPwTkrBK6kyCOc6X
User asked: 'Could you suggest a gluten-free lunch recipe?'
Run finished with status: RunStatus.COMPLETED
User asked: 'Show me some heart-healthy meal ideas.'
Run finished with status: RunStatus.COMPLETED
User asked: 'What guidelines do you have for someone with diabetes?'
Run finished with status: RunStatus.COMPLETED


### 5.3.1 Viewing the conversation
We can retrieve the conversation messages to see how the agent responded, check if it cited file passages, etc.

In [14]:
def display_thread(thread_id):
    try:
        messages = project_client.agents.list_messages(thread_id=thread_id)
        print("\n🗣️ Conversation:")
        for m in reversed(messages.data):
            if m.content:
                last_content = m.content[-1]
                if hasattr(last_content, "text"):
                    print(f"[{m.role.upper()}]: {last_content.text.value}\n")

        print("\n📎 Checking for citations...")
        for c in messages.file_citation_annotations:
            print(f"- Citation snippet: '{c.text}' from file ID: {c.file_citation['file_id']}")
    except Exception as e:
        print(f"❌ Could not display thread: {e}")

# If we created a thread above, let's read it
if health_agent and thread:
    display_thread(thread.id)


🗣️ Conversation:
[USER]: Could you suggest a gluten-free lunch recipe?

[ASSISTANT]: Absolutely! Here’s a simple gluten-free lunch recipe idea:

**Quinoa & Chickpea Salad**

**Ingredients:**
- 1 cup cooked quinoa (gluten-free)
- 1 cup canned chickpeas, rinsed and drained
- 1 cup cherry tomatoes, halved
- 1/2 cucumber, diced
- 1/4 cup chopped red onion
- 1/4 cup feta cheese (optional)
- 2 tbsp olive oil
- Juice of 1 lemon
- Salt and pepper to taste
- Fresh parsley or mint to garnish

**Instructions:**
1. In a large bowl, combine quinoa, chickpeas, tomatoes, cucumber, red onion, and feta cheese.
2. In a small bowl, whisk together olive oil, lemon juice, salt, and pepper.
3. Pour the dressing over the salad and toss well.
4. Garnish with chopped herbs.
5. Serve chilled or at room temperature.

This meal is naturally gluten-free, packed with protein, and very easy to prepare. For added flavor, try adding diced avocado or roasted sweet potatoes.

**Disclaimer:** I am not a medical professi

# 6. Cleanup
If desired, we can remove the vector store, files, and agent to keep things tidy. (In a real solution, you might keep them around.)

In [ ]:
def cleanup_resources():
    try:
        if 'vector_store' in globals() and vector_store:
            project_client.agents.delete_vector_store(vector_store.id)
            print("🗑️ Deleted vector store.")

        if 'file_ids' in globals() and file_ids:
            for fid in file_ids:
                project_client.agents.delete_file(fid)
            print("🗑️ Deleted uploaded files.")

        if 'health_agent' in globals() and health_agent:
            project_client.agents.delete_agent(health_agent.id)
            print("🗑️ Deleted health agent.")

        if 'sample_files' in globals() and sample_files:
            for sf in sample_files:
                if os.path.exists(sf):
                    os.remove(sf)
            print("🗑️ Deleted local sample files.")
    except Exception as e:
        print(f"❌ Error cleaning up: {e}")


cleanup_resources()

# 🎉 Wrap-Up
We've demonstrated:
1. **Basic LLM calls** with `AIProjectClient`.
2. **Listing connections** in your Azure AI Foundry project.
3. **Observability & tracing** in both local (console, OTLP endpoint) and cloud (App Insights) contexts.
4. A quick **Agent** scenario that uses a vector store for searching sample docs.

## Next Steps
- Check the **Tracing** tab in your Azure AI Foundry portal to see the logs.
- Explore advanced queries in Application Insights.
- Use [Prompty](https://github.com/microsoft/prompty) or [Aspire](https://learn.microsoft.com/dotnet/aspire/) for local telemetry dashboards.
- Incorporate this approach into your **production** GenAI pipelines!

> 🏋️ **Health Reminder**: The LLM's suggestions are for demonstration only. For real health decisions, consult a professional.

Happy Observing & Tracing! 🎉